<a href="https://colab.research.google.com/github/jpdhanam/Jaya-priya-J-.devtern-fake-news-detection/blob/main/House_Price_Prediction_and_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'usa-housing:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4909%2F7459%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240915%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240915T155745Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D20ea966e6969da94582dc5141e0cedb3ba8175fb4dad7820d426b7cff1caed4f93b1d268f38632eb67dc97e64c284e196c456ed2d86620e2cff24603b961ddb6b09dfa982fa74a91a23f06b5924519fbfcc7f4a6a00cc2a7d57edb28220cc9922f7af1628fff1ea9d3d5273207afc5a0053667c4472c15e96fab1141098818a1ba012e506419cb13c9dec5676cb64b79fe9fa9508402fca34f6a2ae28817254263f0936399defc02d5377a05f1e0c4dc9a607f79b44e1531a80b3a92d02cf8921d494023d2b539f0ced0c6ba740b71d360eb59ffe133bd526221f9c5600e31029390a88252d3eb09db739fc16c282854a44c6f2022cb9542c2d3304abf36c4b2'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df=pd.read_csv('/kaggle/input/usa-housing/USA_Housing.csv')
df

In [ ]:
#Provide statistical information about the dataset
df.describe()

In [ ]:
df.info()

In [ ]:
sns.pairplot(df)

In [ ]:
#Plotting the distribution of target column(which is 'Price')
sns.distplot(df['Price'])

In [ ]:
df.columns

In [ ]:
X=df.drop(['Price','Address'], axis=1)

In [ ]:
#Teaget variable
y=df['Price']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Splitting the data in training and testing dataset
X_test,X_train,y_train,y_test = train_test_split(X,y,test_size=0.5,random_state = 42)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
print(lr.intercept_)

In [ ]:
lr.coef_

In [ ]:
cdf = pd.DataFrame(lr.coef_,X.columns,columns=['Coeff'])

In [ ]:
cdf

In [ ]:
#The output may not make sense as the dataset is artificially collected

In [ ]:
#predicting on the unseen dataset(Test dataset)
predict = lr.predict(X_test)
#Predicted prices of the house
predict

In [ ]:
plt.scatter(y_test,predict)

In [ ]:
#Evaluation of model performance
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
mean_absolute_error(y_test,predict)

In [ ]:
mean_squared_error(y_test,predict)